In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [2]:
import sys
sys.getdefaultencoding()

'utf-8'

Open the questionnaire file (word document version of the questionnaire saved as html):

In [3]:
questionnaire_file = open("FF_f4_Questionnaire.htm", "r") 

In [4]:
soup=BeautifulSoup(questionnaire_file, "html.parser")

In [5]:
ps=[p for p in soup.find_all('p')]

In [6]:
ps[108:111]

[<p class="MsoNormal"><span style='font-family:"Arial",sans-serif'> </span></p>,
 <p class="MsoNormal" style="margin-left:.5in;text-indent:.5in"><span style='font-family:"Arial",sans-serif'>|___|___| NIGHTS</span></p>,
 <p class="MsoNormal"><span style='font-family:"Arial",sans-serif'> </span></p>]

Opening the excel file with all the questions for p6, using the pandas python library:

In [7]:
df=pd.read_excel("FFCf4.xlsx") #this is an excel file with all the questions for p6

In [8]:
df.shape

(1035, 3)

In [9]:
reg = re.compile(r'(?P<section>[A-Za-z]*\d*)(?P<rest>.*)')

def format_q_num(name):
    """takes in the question numbers in the name column and outputs them in the format
    they appear in the text"""
    name=name.lstrip('f4').split('_')[0] 
    name=reg.search(name)
    section=name.group('section').upper()
    rest=name.group('rest').upper()
    return section+rest

In [10]:
df['question_number']=df.name.apply(lambda x: format_q_num(x))

In [11]:
df['multiple_answers']=df.name.apply(lambda x: 1 if "_" in x else 0) 
#adds a column where 1 indicates that it is one of the questions with multiple entries in the data frame
#I think these are mostly (only?) the "select all that apply" questions

In [12]:
df.head(20)

,position,name,varlab,question_number,multiple_answers
0,6673,f4a2,How much of the time does child spend with you?,A2,0
1,6674,f4a2c,Has child stayed with you in past year?,A2C,0
2,6675,f4a2d,How many nights has child spent with you in pa...,A2D,0
3,6676,f4a3,How many months ago did child stop living with...,A3,0
4,6677,f4a3a1,What is main reason child stopped living with ...,A3A1,0
5,6678,f4a3a2,Who does child usually live with?,A3A2,0
6,6679,f4a3b,Are child's foster parents related to you?,A3B,0
7,6680,f4a3b1a,Does primary caregiver receive payment?,A3B1A,0
8,6681,f4a3b1b_1,Who provides payment: father?,A3B1B,1
9,6682,f4a3b1b_2,Who provides payment: mother?,A3B1B,1


In [13]:
def find_question(q_number):
    """finds where the question number appears in the text - only if it only appears once"""
    ps_text = []
    ps_idx = []
    ps_startswith = []
    for idx, p in enumerate(ps):
        if q_number+"." in p.get_text():
            if p.get_text().lstrip().startswith(q_number+"."):
                ps_text.append(p.get_text().lstrip())
                ps_idx.append(idx)
    if len(ps_idx)!=1:
        return np.nan, np.nan
    else:
        return ps_text[0].replace("\n"," ").strip(), ps_idx[0]

In [14]:
a2 = ps[83:95]
a2[0].get_text().lstrip().startswith('A2.')

True

In [15]:
ps[151].get_text().lstrip().startswith('A2.')

False

In [16]:
def find_probe(q_number):
    probe = re.findall('[A-Z]{,1}\d{,2}', q_number)[0]
    return find_question(probe)[0]

In [17]:
for i in range(len(df.question_number)):
    q_number = df.loc[i,'question_number']
    if re.match('^[A-Z]{,1}\d{,2}', q_number):
        df.loc[i,'probe'] = find_probe(q_number)
    else:
        df.loc[i,'probe'] = "None"

In [18]:
df['ps_text'], df['ps_idx'] = zip(*df['question_number'].map(find_question))

In [19]:
df

,position,name,varlab,question_number,multiple_answers,probe,ps_text,ps_idx
0,6673,f4a2,How much of the time does child spend with you?,A2,0,A2. How much of the time does (CHILD...,A2. How much of the time does (CHILD...,83.0
1,6674,f4a2c,Has child stayed with you in past year?,A2C,0,A2. How much of the time does (CHILD...,A2C. Has (he/she) stayed with you in th...,101.0
2,6675,f4a2d,How many nights has child spent with you in pa...,A2D,0,A2. How much of the time does (CHILD...,A2D. How many nights has (CHILD) spent ...,107.0
3,6676,f4a3,How many months ago did child stop living with...,A3,0,A3. How many months ago did (he/she)...,A3. How many months ago did (he/she)...,112.0
4,6677,f4a3a1,What is main reason child stopped living with ...,A3A1,0,A3. How many months ago did (he/she)...,A3A1. What was the main reason (he/she) s...,120.0
5,6678,f4a3a2,Who does child usually live with?,A3A2,0,A3. How many months ago did (he/she)...,A3A2. CODE WITHOUT ASKING IF KNOWN.,151.0
6,6679,f4a3b,Are child's foster parents related to you?,A3B,0,A3. How many months ago did (he/she)...,A3B. Are (CHILD's) foster parents relat...,173.0
7,6680,f4a3b1a,Does primary caregiver receive payment?,A3B1A,0,A3. How many months ago did (he/she)...,A3B1A. Does (PERSON IN A3A2) receive any kin...,181.0
8,6681,f4a3b1b_1,Who provides payment: father?,A3B1B,1,A3. How many months ago did (he/she)...,A3B1B. Who provides this payment?,189.0
9,6682,f4a3b1b_2,Who provides payment: mother?,A3B1B,1,A3. How many months ago did (he/she)...,A3B1B. Who provides this payment?,189.0


In [20]:
np.isnan(df.ps_idx).sum() #number of questions where the question number doesn't appear or appears more than once

311

In [21]:
print(df.ps_text[11])
print(df.ps_idx[11])

A3B1B.   Who provides this payment?
189.0


In [22]:
#get index of <p> tag for the next question so we can split the questions (can't just shift because some questions have more than one answer or missing values)
all_idx=np.array(df.ps_idx)
all_idx=all_idx[~np.isnan(all_idx)]
all_idx=np.append(all_idx, [len(ps)])
df['ps_idx_next']=df.ps_idx.apply(lambda x: all_idx[all_idx > x].min() if ~np.isnan(x) else np.nan)

In [23]:
consequitivedots = re.compile(r'\.{3,}')

In [24]:
def clean_text(string):
    return string.replace("\n"," ").strip()

def get_answers(start_idx,stop_idx):
    if np.isnan(start_idx) or np.isnan(stop_idx):
        return ([],0)
    else:
        answers = [p for p in ps[int(start_idx):int(stop_idx)] if len(p.find_all(string = consequitivedots))>0]
        answers = [(clean_text(a.get_text().split(".")[0]),clean_text(a.get_text().split(".")[-1])) for a in answers]
        return(answers,len(answers)) 

In [25]:
df['answers'], df['n_answers'] = zip(*map(get_answers, df['ps_idx'], df['ps_idx_next']))

In [26]:
x = 0
while x < len(df.answers):
    ans = df.answers[x]
    for i in range(len(ans)): #for each answer choice,
        s = ans[i][1]        # store number string
        a = re.sub('è',' --> ', s) # sub the arrow in for skip pattern
        t = (ans[i][0],a) # create a new tuple with arrow
        ans[i] = t
    for y in range(len(ans)):
        df.loc[x,'val{0}'.format(y)] = ans[y][1] #create new 'val(y)' column
        df.loc[x,'lab{0}'.format(y)] = ans[y][0] #create new 'lab(y)' column
        
    x += 1

In [27]:
df.head(10)

,position,name,varlab,question_number,multiple_answers,probe,ps_text,ps_idx,ps_idx_next,answers,...,val95,lab95,val96,lab96,val97,lab97,val98,lab98,val99,lab99
0,6673,f4a2,How much of the time does child spend with you?,A2,0,A2. How much of the time does (CHILD...,A2. How much of the time does (CHILD...,83.0,101.0,"[(All or most of the time,, 1 --> GO TO A4),...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6674,f4a2c,Has child stayed with you in past year?,A2C,0,A2. How much of the time does (CHILD...,A2C. Has (he/she) stayed with you in th...,101.0,107.0,"[(YES, 1), (NO, 2 --> GO TO A3A1)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6675,f4a2d,How many nights has child spent with you in pa...,A2D,0,A2. How much of the time does (CHILD...,A2D. How many nights has (CHILD) spent ...,107.0,112.0,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6676,f4a3,How many months ago did child stop living with...,A3,0,A3. How many months ago did (he/she)...,A3. How many months ago did (he/she)...,112.0,120.0,"[(THE TIME, -10 --> GO TO A3A2)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6677,f4a3a1,What is main reason child stopped living with ...,A3A1,0,A3. How many months ago did (he/she)...,A3A1. What was the main reason (he/she) s...,120.0,151.0,"[(LEGALLY LOST CUSTODY, 1), (AGENCY/COURT REMO...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6678,f4a3a2,Who does child usually live with?,A3A2,0,A3. How many months ago did (he/she)...,A3A2. CODE WITHOUT ASKING IF KNOWN.,151.0,173.0,"[(BIOLOGICAL MOTHER, 1 --> GO TO A3C), (MATE...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6679,f4a3b,Are child's foster parents related to you?,A3B,0,A3. How many months ago did (he/she)...,A3B. Are (CHILD's) foster parents relat...,173.0,181.0,"[(YES, 1), (NO, 2)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6680,f4a3b1a,Does primary caregiver receive payment?,A3B1A,0,A3. How many months ago did (he/she)...,A3B1A. Does (PERSON IN A3A2) receive any kin...,181.0,189.0,"[(YES, 1), (NO, 2), (DON’T KNOW, -2), (REFUSED...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6681,f4a3b1b_1,Who provides payment: father?,A3B1B,1,A3. How many months ago did (he/she)...,A3B1B. Who provides this payment?,189.0,203.0,"[(A3B1B_2 CHILD’S MOTHER, 2), (A3B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6682,f4a3b1b_2,Who provides payment: mother?,A3B1B,1,A3. How many months ago did (he/she)...,A3B1B. Who provides this payment?,189.0,203.0,"[(A3B1B_2 CHILD’S MOTHER, 2), (A3B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
i = 0
while i < len(df.ps_text):
    if df.multiple_answers[i] == 1:
        df.loc[i,'probe'] = df.loc[i,'ps_text']
        values = df.loc[i,'val0':].tolist()
        part = (df.name[i].lstrip('f4').split('_')[1])
        match = re.match('[0-9]{1,3}',part)
        if match:
            if part in values:
                index = values.index(part) +1
                text = values[index]
                df.loc[i,'ps_text'] = text
    i += 1

In [29]:
print(df.ps_text[11])

A3B1B_4             FRIEND


In [30]:
df.answers[11]

[('A3B1B_2\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 CHILD’S MOTHER',
  '2'),
 ('A3B1B_3\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 OTHER RELATIVE OF CHILD',
  '3'),
 ('A3B1B_4\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 FRIEND', '4'),
 ('A3B1B_5\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 GOVERNMENT AGENCY',
  '5'),
 ('A3B1B_6\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 OTHER (NOT SPECIFIED)',
  '6')]

In [31]:
df.head(20)

,position,name,varlab,question_number,multiple_answers,probe,ps_text,ps_idx,ps_idx_next,answers,...,val95,lab95,val96,lab96,val97,lab97,val98,lab98,val99,lab99
0,6673,f4a2,How much of the time does child spend with you?,A2,0,A2. How much of the time does (CHILD...,A2. How much of the time does (CHILD...,83.0,101.0,"[(All or most of the time,, 1 --> GO TO A4),...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6674,f4a2c,Has child stayed with you in past year?,A2C,0,A2. How much of the time does (CHILD...,A2C. Has (he/she) stayed with you in th...,101.0,107.0,"[(YES, 1), (NO, 2 --> GO TO A3A1)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6675,f4a2d,How many nights has child spent with you in pa...,A2D,0,A2. How much of the time does (CHILD...,A2D. How many nights has (CHILD) spent ...,107.0,112.0,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6676,f4a3,How many months ago did child stop living with...,A3,0,A3. How many months ago did (he/she)...,A3. How many months ago did (he/she)...,112.0,120.0,"[(THE TIME, -10 --> GO TO A3A2)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6677,f4a3a1,What is main reason child stopped living with ...,A3A1,0,A3. How many months ago did (he/she)...,A3A1. What was the main reason (he/she) s...,120.0,151.0,"[(LEGALLY LOST CUSTODY, 1), (AGENCY/COURT REMO...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6678,f4a3a2,Who does child usually live with?,A3A2,0,A3. How many months ago did (he/she)...,A3A2. CODE WITHOUT ASKING IF KNOWN.,151.0,173.0,"[(BIOLOGICAL MOTHER, 1 --> GO TO A3C), (MATE...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6679,f4a3b,Are child's foster parents related to you?,A3B,0,A3. How many months ago did (he/she)...,A3B. Are (CHILD's) foster parents relat...,173.0,181.0,"[(YES, 1), (NO, 2)]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6680,f4a3b1a,Does primary caregiver receive payment?,A3B1A,0,A3. How many months ago did (he/she)...,A3B1A. Does (PERSON IN A3A2) receive any kin...,181.0,189.0,"[(YES, 1), (NO, 2), (DON’T KNOW, -2), (REFUSED...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6681,f4a3b1b_1,Who provides payment: father?,A3B1B,1,A3B1B. Who provides this payment?,A3B1B. Who provides this payment?,189.0,203.0,"[(A3B1B_2 CHILD’S MOTHER, 2), (A3B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6682,f4a3b1b_2,Who provides payment: mother?,A3B1B,1,A3B1B. Who provides this payment?,A3B1B_2 CHILD’S MOTHER,189.0,203.0,"[(A3B1B_2 CHILD’S MOTHER, 2), (A3B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df.drop(['answers','n_answers','multiple_answers','question_number','ps_idx','ps_idx_next'],axis=1,inplace=True)

In [33]:
df.head(20)

,position,name,varlab,probe,ps_text,val0,lab0,val1,lab1,val2,...,val95,lab95,val96,lab96,val97,lab97,val98,lab98,val99,lab99
0,6673,f4a2,How much of the time does child spend with you?,A2. How much of the time does (CHILD...,A2. How much of the time does (CHILD...,1 --> GO TO A4,"All or most of the time,",2 --> GO TO A3F,"About half of the time,",3 --> GO TO A2D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6674,f4a2c,Has child stayed with you in past year?,A2. How much of the time does (CHILD...,A2C. Has (he/she) stayed with you in th...,1,YES,2 --> GO TO A3A1,NO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6675,f4a2d,How many nights has child spent with you in pa...,A2. How much of the time does (CHILD...,A2D. How many nights has (CHILD) spent ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6676,f4a3,How many months ago did child stop living with...,A3. How many months ago did (he/she)...,A3. How many months ago did (he/she)...,-10 --> GO TO A3A2,THE TIME,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6677,f4a3a1,What is main reason child stopped living with ...,A3. How many months ago did (he/she)...,A3A1. What was the main reason (he/she) s...,1,LEGALLY LOST CUSTODY,2,AGENCY/COURT REMOVED CHILD,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6678,f4a3a2,Who does child usually live with?,A3. How many months ago did (he/she)...,A3A2. CODE WITHOUT ASKING IF KNOWN.,1 --> GO TO A3C,BIOLOGICAL MOTHER,2,MATERNAL GRANDPARENT(S),3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6679,f4a3b,Are child's foster parents related to you?,A3. How many months ago did (he/she)...,A3B. Are (CHILD's) foster parents relat...,1,YES,2,NO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6680,f4a3b1a,Does primary caregiver receive payment?,A3. How many months ago did (he/she)...,A3B1A. Does (PERSON IN A3A2) receive any kin...,1,YES,2,NO,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,6681,f4a3b1b_1,Who provides payment: father?,A3B1B. Who provides this payment?,A3B1B. Who provides this payment?,2,A3B1B_2 CHILD’S MOTHER,3,A3B1B_3 OTHER RELATIVE OF CHILD,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6682,f4a3b1b_2,Who provides payment: mother?,A3B1B. Who provides this payment?,A3B1B_2 CHILD’S MOTHER,2,A3B1B_2 CHILD’S MOTHER,3,A3B1B_3 OTHER RELATIVE OF CHILD,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df.to_csv(path_or_buf = "f4Qtext.csv")